In [8]:
from langgraph.graph import StateGraph,START, END
from typing import TypedDict, Annotated, Literal
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")

In [15]:
class ReviewState(TypedDict):
    review: str
    semantic_feedback: Literal["positive", "negative"]
    diagnosis: dict
    response: str

class SemanticFeedback(BaseModel):
    feedback: Literal["positive", "negative"] = Field(..., description="Indicates whether the review is positive or negative.")

class Diagnosis(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

structured_output_model = model.with_structured_output(SemanticFeedback)
structured_output_diagnosis_model = model.with_structured_output(Diagnosis)

def get_semantic_feedback(state: ReviewState) -> dict:
    prompt = PromptTemplate(
        input_variables=["review"],
        template="Analyze the following review and determine if it is positive or negative:\n\n{review}\n\nRespond with 'positive' or 'negative'."
    )
    response = structured_output_model.invoke(prompt.format_prompt(review=state['review']).to_string())
    return {'semantic_feedback': response.feedback}

def get_diagnosis(state: ReviewState) -> dict:
    prompt = PromptTemplate(
        input_variables=["review"],
        template="Analyze the following review and determine the issue type, tone, and urgency:\n\n{review}\n\nRespond with a JSON object containing 'issue_type', 'tone', and 'urgency'."
    )
    response = structured_output_diagnosis_model.invoke(prompt.format_prompt(review=state['review']).to_string())
    return {'diagnosis': {
        "issue_type": response.issue_type,
        "tone": response.tone,
        "urgency": response.urgency
    }}

def get_positive_response(state: ReviewState) -> dict:
    prompt = PromptTemplate(
        input_variables=["review"],
        template="The following is a positive customer review:\n\n{review}\n\nWrite a short, warm thank-you response acknowledging their feedback."
    )
    response = model.invoke(prompt.format_prompt(review=state['review']).to_string())
    return {'response': response.content}

def check_sentiment(state: ReviewState) -> Literal["get_positive_response", "get_diagnosis"]:
    if state['semantic_feedback'] == "positive":
        return "get_positive_response"
    else:
        return "get_diagnosis"
    

def get_negative_response(state: ReviewState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.
"""
    response = model.invoke(prompt).content

    return {'response': response}

In [16]:
graph = StateGraph(ReviewState)

graph.add_node("get_semantic_feedback", get_semantic_feedback)
graph.add_node("get_diagnosis", get_diagnosis)
graph.add_node("get_positive_response", get_positive_response)
graph.add_node("get_negative_response", get_negative_response)

graph.add_edge(START, "get_semantic_feedback")
graph.add_conditional_edges("get_semantic_feedback", check_sentiment)
graph.add_edge("get_positive_response", END)
graph.add_edge("get_diagnosis", "get_negative_response")
graph.add_edge("get_negative_response", END)

compiled_graph = graph.compile()

result = compiled_graph.invoke({
    "review": "The product quality is outstanding and exceeded my expectations!"
})
print(result)

{'review': 'The product quality is outstanding and exceeded my expectations!', 'semantic_feedback': 'positive', 'response': "Dear [Customer's Name],\n\nThank you so much for your kind words! We're thrilled to hear that the product exceeded your expectations. Your satisfaction means the world to us! If you have any more feedback or need assistance, please don't hesitate to reach out.\n\nWarm regards,  \n[Your Name]  \n[Your Company]  "}
